In [ ]:
#########################################################################
################### OBTENER LA BASE DE DATOS ############################
#########################################################################

# Descargar las imágenes de http://www.vision.caltech.edu/visipedia/CUB-200.html
# Descomprimir el fichero.
# Descargar también el fichero list.tar.gz, descomprimirlo y guardar los ficheros
# test.txt y train.txt dentro de la carpeta de imágenes anterior. Estos 
# dos ficheros contienen la partición en train y test del conjunto de datos.

##### EN CASO DE USAR COLABORATORY
# Sube tanto las imágenes como los ficheros text.txt y train.txt a tu drive.
# Después, ejecuta esta celda y sigue las instrucciones para montar 
# tu drive en colaboratory.
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# -*- coding: utf-8 -*-

#########################################################################
################ CARGAR LAS LIBRERÍAS NECESARIAS ########################
#########################################################################

# Terminar de rellenar este bloque con lo que vaya haciendo falta

# Importar librerías necesarias
import numpy as np
import keras
import keras.utils as np_utils
from keras.preprocessing.image import load_img,img_to_array
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy

# Importar el optimizador a usar
from keras.optimizers import SGD

# Importar modelos y capas específicas que se van a usar
from keras.models import Model, Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D
from keras.layers import Flatten, Dropout, BatchNormalization, Activation


# Importar el modelo ResNet50 y su respectiva función de preprocesamiento,
# que es necesario pasarle a las imágenes para usar este modelo
from keras.applications.resnet50 import ResNet50, preprocess_input


# Importar el optimizador a usar
from keras.optimizers import SGD

#Path donde se guardan las imagenes y los archivos train y test
PATH= "/content/drive/MyDrive/imagenes"

#########################################################################
################## FUNCIÓN PARA LEER LAS IMÁGENES #######################
#########################################################################

# Dado un fichero train.txt o test.txt y el path donde se encuentran los
# ficheros y las imágenes, esta función lee las imágenes
# especificadas en ese fichero y devuelve las imágenes en un vector y 
# sus clases en otro.

def leerImagenes(vec_imagenes, path):
  clases = np.array([img.split('/')[0] for img in vec_imagenes])
  imagenes = np.array([img_to_array(load_img(path + "/" + img, 
                                             target_size = (224, 224))) 
                       for img in vec_imagenes])
  return imagenes, clases

#########################################################################
############# FUNCIÓN PARA CARGAR EL CONJUNTO DE DATOS ##################
#########################################################################

# Usando la función anterior, y dado el path donde se encuentran las
# imágenes y los archivos "train.txt" y "test.txt", devuelve las 
# imágenes y las clases de train y test para usarlas con keras
# directamente.

def cargarDatos(path):
  # Cargamos los ficheros
  train_images = np.loadtxt(path + "/train.txt", dtype = str)
  test_images = np.loadtxt(path + "/test.txt", dtype = str)
  
  # Leemos las imágenes con la función anterior
  train, train_clases = leerImagenes(train_images,path)
  test, test_clases = leerImagenes(test_images,path)
  
  # Pasamos los vectores de las clases a matrices 
  # Para ello, primero pasamos las clases a números enteros
  clases_posibles = np.unique(np.copy(train_clases))
  for i in range(len(clases_posibles)):
    train_clases[train_clases == clases_posibles[i]] = i
    test_clases[test_clases == clases_posibles[i]] = i

  # Después, usamos la función to_categorical()
  train_clases = np_utils.to_categorical(train_clases, 200)
  test_clases = np_utils.to_categorical(test_clases, 200)
  
  # Barajar los datos
  train_perm = np.random.permutation(len(train))
  train = train[train_perm]
  train_clases = train_clases[train_perm]

  test_perm = np.random.permutation(len(test))
  test = test[test_perm]
  test_clases = test_clases[test_perm]
  
  return train, train_clases, test, test_clases

#########################################################################
######## FUNCIÓN PARA OBTENER EL ACCURACY DEL CONJUNTO DE TEST ##########
#########################################################################

# Esta función devuelve el accuracy de un modelo, definido como el 
# porcentaje de etiquetas bien predichas frente al total de etiquetas.
# Como parámetros es necesario pasarle el vector de etiquetas verdaderas
# y el vector de etiquetas predichas, en el formato de keras (matrices
# donde cada etiqueta ocupa una fila, con un 1 en la posición de la clase
# a la que pertenece y 0 en las demás).

def calcularAccuracy(labels, preds):
  labels = np.argmax(labels, axis = 1)
  preds = np.argmax(preds, axis = 1)
  
  accuracy = sum(labels == preds)/len(labels)
  
  return accuracy

#########################################################################
## FUNCIÓN PARA PINTAR LA PÉRDIDA Y EL ACCURACY EN TRAIN Y VALIDACIÓN ###
#########################################################################

# Esta función pinta dos gráficas, una con la evolución de la función
# de pérdida en el conjunto de train y en el de validación, y otra
# con la evolución del accuracy en el conjunto de train y en el de
# validación. Es necesario pasarle como parámetro el historial
# del entrenamiento del modelo (lo que devuelven las funciones
# fit() y fit_generator()).

def mostrarEvolucion(hist):

  loss = hist.history['loss']
  val_loss = hist.history['val_loss']
  plt.plot(loss)
  plt.plot(val_loss)
  plt.legend(['Training loss', 'Validation loss'])
  plt.show()

  acc = hist.history['accuracy']
  val_acc = hist.history['val_accuracy']
  plt.plot(acc)
  plt.plot(val_acc)
  plt.legend(['Training accuracy', 'Validation accuracy'])
  plt.show()

"""## Usar ResNet50 preentrenada en ImageNet como un extractor de características"""

# Las características extraídas en el paso anterior van a ser la entrada
# de un pequeño modelo de dos capas Fully Conected, donde la última será la que 
# nos clasifique las clases de Caltech-UCSD (200 clases). De esta forma, es 
# como si hubiéramos fijado todos los parámetros de ResNet50 y estuviésemos
# entrenando únicamente las capas añadidas. Definir dicho modelo.
"""Función para compilar el modelo. De optimizador elegimos SGD."""
def compilar(model):

    # Definimos el optimizador
    opt = SGD(lr = 0.01, decay = 1e-6,
              momentum = 0.9, nesterov = True)

    # Compilamos el modelo
    model.compile(loss = categorical_crossentropy,
                  optimizer = opt,
                  metrics = ['accuracy'])
    return model
"""Función para entrenar el modelo segú si se le pasa un datagen o no"""
def entrenarModelo(model, x_train, y_train, datagen="foo", batch_size=64, epocas=12):

    if(datagen=="foo"):
        #Entrenamos el modelo de forma normal si no hay datagen
        historial = model.fit(x_train, y_train,
                              validation_split=0.1,
                              batch_size=batch_size,epochs=epocas,verbose=1)
    else:
        #Generamos el entrenamiento para fit_generator
        generator_train=datagen.flow(x_train, y_train,
                     batch_size = batch_size, subset ='training')   
        #Generamos la validación para fit_generator
        generator_valid=datagen.flow(x_train, y_train,
                     batch_size = batch_size, subset ='validation')
        #Entrenamos el modelo
        historial=model.fit_generator(generator_train,
                            steps_per_epoch = len(x_train)*0.9/batch_size,
                            epochs = epocas, 
                            validation_data = generator_valid, 
                            validation_steps = len(x_train)*0.1/batch_size)
    #Devolvemos el historial de entrenamiento
    return historial



"""Evalua el modelo con el conjunto test"""
def prediccionTest(model, x_test, y_test):
    #Evaluamos el modelo
    score = model.evaluate(x_test, y_test, verbose=0)
    return score


"""Función para definir el modelo y entrenarlo, así como para obtener su
evaluación sobre el conjunto de test"""
def generar_modelo(modelo_gen, x_train, y_train, x_test, y_test, epocas=15):

    #ĆCompilamos el modelo
    modelo_gen=compilar(modelo_gen)

    #Mostramos el modelo
    print(modelo_gen.summary())

    #Entrenamiento del modelo
    hist = entrenarModelo(modelo_gen, x_train, y_train, epocas=epocas)

    #Evaluacion del modelo
    score = prediccionTest(modelo_gen, x_test, y_test)

    return score, hist

"""Modelo simple para comparar con el FC y reentrenado"""
def modelo_simple():
    model=Sequential()
    model.add(Dense(200, activation = 'softmax', input_shape=(7,7,2048)))
    return model

"""Modelo con capas FC patra reentrenar"""
def modelo_fc():
    model = Sequential()
    model.add(Dense(1024,activation = 'relu',input_shape =(2048,)))
    model.add(Dense(512, activation = 'relu' ))
    model.add(Dropout(0.25))
    model.add(Dense(200, activation='softmax'))

    return model


# Definir el modelo ResNet50 (preentrenado en ImageNet y sin la última capa).
def resnet_extractor_car(apartadoA2):
    #Para establecer un modelo simple o uno FC, reentrenar y comparar
    if apartadoA2:
        modelo_gen = modelo_simple()
    else:
        modelo_gen = modelo_fc()
    
    score, historial=generar_modelo(modelo_gen,preds_train, y_train, 
                                    preds_test, y_test)
    mostrarEvolucion(historial)
    #Mostramos la accuracy
    print('Test accuracy:', score[1])
    #Mostramos la pérdida
    print('Test loss:', score[0])

#Apartado 1.B
"""Modelo usado en el apartado B"""
def modeloB():
  model = Sequential()
  model.add(Conv2D(1024,
                     kernel_size = (3, 3),
                     input_shape = (7,7,2048)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dropout(0.25))
  model.add(Dense(512, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Dense(218, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(200, activation='softmax'))
  return model
"""Función para el apartado B"""
def resnet_extractor_carB():
    modelo_gen = modeloB()
    modelo_gen=compilar(modelo_gen)
    score, historial=generar_modelo(modelo_gen,preds_train, y_train, 
                                    preds_test, y_test)
    mostrarEvolucion(historial)
    #Mostramos la accuracy
    print('Test accuracy:', score[1])
    #Mostramos la pérdida
    print('Test loss:', score[0])

"""Ejecucion del primer apartado"""

print("Leyendo imágenes...\n")
x_train, y_train, x_test, y_test = cargarDatos(PATH)

#Creamos un ImageDataGenerator para train y otro para test
datagen_train = ImageDataGenerator(preprocessing_function = preprocess_input)
datagen_test = ImageDataGenerator(preprocessing_function = preprocess_input)

#Definición del modelo preentrenado sin FC ni la última capa
resnet50 = ResNet50(include_top= False, weights= 'imagenet', pooling= 'avg')

#Extraemos las caracteristicas con predict_generator del modelo resnet
print("Extrayendo características para el apartado A...\n")
preds_train = resnet50.predict_generator(datagen_train.flow(x_train,
                                              batch_size = 1,
                                              shuffle = False),
                                verbose = 1, steps = len(x_train))
preds_test = resnet50.predict_generator(datagen_train.flow(x_test,
                                              batch_size = 1,
                                              shuffle = False),
                                verbose = 1, steps = len(x_test))

#Usando modelo simple
resnet_extractor_car(False)
input("\n--- Pulsar tecla para continuar. Comparacion con modelo Simple ---\n")
#Usando modelo fully-connected con 2 capas más
resnet_extractor_car(True)


#Extraemos las caracteristicas con predict_generator del modelo resnet
input("\n--- Pulsar tecla para continuar. Apartado B ---\n")
print("Extrayendo características para el apartado B...\n")
resnet50 = ResNet50(include_top= False, weights= 'imagenet', pooling = None)
preds_train = resnet50.predict_generator(datagen_train.flow(x_train,
                                              batch_size = 1,
                                              shuffle = False),
                                verbose = 1, steps = len(x_train))
preds_test = resnet50.predict_generator(datagen_train.flow(x_test,
                                              batch_size = 1,
                                              shuffle = False),
                                verbose = 1, steps = len(x_test))

resnet_extractor_carB()

###########################################
"""## Reentrenar ResNet50 (fine tunning)"""
###########################################

# Añadir nuevas capas al final de ResNet50 (recuerda que es una instancia de
# la clase Model).
"""Modelo para el apartado 2"""
def modeloFT(modelo):
    modelo = Dense(1024, activation = 'relu') (modelo) 
    modelo = Dropout(0.5) (modelo)
    modelo = Dense(200, activation = 'softmax') (modelo)

    return modelo

# Definir un objeto de la clase ImageDataGenerator para train y otro para test
# con sus respectivos argumentos.
def resnetFT():
  datagen_train = ImageDataGenerator(preprocessing_function = preprocess_input, validation_split=0.1)
  datagen_test = ImageDataGenerator(preprocessing_function = preprocess_input)
  #Modelo para la entrada
  resnet50 = ResNet50(include_top= False, weights= 'imagenet', pooling= 'avg', input_shape = (224,224,3))
  output = modeloFT(resnet50.output)
  modelo = Model(inputs = resnet50.input, outputs = output)
  compilar(modelo)
  historial=entrenarModelo(modelo, x_train, y_train, datagen=datagen_train)
  score = modelo.evaluate_generator(datagen_test.flow(x_test,
                                                  y_test,
                                                  batch_size = 1,
                                                  shuffle = False),
                                      verbose = 0,
                                      steps = len(x_test))
  mostrarEvolucion(historial)
  #Mostramos la accuracy
  print('Test accuracy:', score[1])
  #Mostramos la pérdida
  print('Test loss:', score[0])

"""Ejecución secuencial para el apartado 2"""
input("\n--- Pulsar tecla para continuar. Apartado 2 ---\n")
resnetFT()
